In [3]:
import numpy as np
import scipy.io as sio
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import matplotlib.pyplot as plt
import os
import re

# Enable GPU if available
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Function to load MAT files and extract metadata
def load_mat_files(folder_path):
    data = {}
    filename_pattern = re.compile(r"roof_type_(\w+)_([0-9]+)_deg([0-9]+)\.mat")

    for file in os.listdir(folder_path):
        if file.endswith(".mat"):
            match = filename_pattern.match(file)
            if match:
                roof_type = match.group(1)  # Extract roof type
                height_ratio = int(match.group(2))  # Extract height ratio
                wind_angle = int(match.group(3))  # Extract wind angle

                mat_data = sio.loadmat(os.path.join(folder_path, file))
                data[(roof_type, height_ratio, wind_angle)] = mat_data
    return data

# Specify the folder containing MAT files
folder_path = "C:/Users/rrsuj/Downloads/capstone/data"
data_dict = load_mat_files(folder_path)

# Extract pressure coefficient values and corresponding metadata
X_train, Y_train = [], []

for (roof_type, height_ratio, wind_angle), mat_data in data_dict.items():
    if 'Cp' in mat_data:
        Cp = mat_data['Cp']  # Extract pressure coefficient matrix

        # Flatten data and store
        X_train.append([height_ratio, wind_angle])  # Height ratio and wind angle as inputs
        Y_train.append(Cp.flatten())  # Pressure coefficient values

# Convert to NumPy arrays
X_train = np.array(X_train).reshape(-1, 2)
Y_train = np.array(Y_train).reshape(-1, 1)

print("Training Data Shape:", X_train.shape, Y_train.shape)

# Physics-Informed Loss (Bernoulli-based correction)
def physics_loss(y_true, y_pred):
    rho = 1.225  # Air density (kg/m³)
    V_ref = 10  # Reference wind speed (m/s)
    Cp_physics = 2 * (y_pred / (rho * V_ref**2))  # Bernoulli-based correction
    return tf.reduce_mean(tf.square(y_true - Cp_physics))

# Define Neural Network Architecture
def build_pinn():
    inputs = Input(shape=(2,))  # Two inputs: height ratio and wind angle
    x = Dense(64, activation="tanh")(inputs)
    x = Dense(64, activation="tanh")(x)
    x = Dense(64, activation="tanh")(x)
    outputs = Dense(1, activation="linear")(x)  # Output mean pressure coefficient
    model = Model(inputs, outputs)
    return model

# Compile the PINN model
pinn = build_pinn()
pinn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=physics_loss)

# Summary of model
pinn.summary()

# Train the PINN model
history = pinn.fit(X_train, Y_train, epochs=500, batch_size=32, verbose=1)

# Plot Training Loss
plt.plot(history.history['loss'], label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training Loss of PINN Model")
plt.show()

# Test wind angles and height ratios (not seen in training data)
test_data = np.array([[12, 15], [18, 37], [6, 60], [12, 75], [18, 95]])

# Predict pressure coefficients
predicted_Cp = pinn.predict(test_data)

# Display Predictions
for i, (height, angle) in enumerate(test_data):
    print(f"Predicted Mean Pressure Coefficient at Height Ratio {height}, Wind Angle {angle}°: {predicted_Cp[i][0]:.4f}")


Training Data Shape: (0, 2) (0, 1)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,577 (33.50 KB)

 Trainable params: 8,577 (33.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500


C:\Users\rrsuj\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ValueError: math domain error